# Installing Sentence_transformers

In [25]:
%pip install -q sentence_transformers

%pip install transformers

%pip install pandas

%pip install django

You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached Django-4.0.4-py3-none-any.whl (8.0 MB)
  Using cached asgiref-3.5.0-py3-none-any.whl (22 kB)
  Using cached sqlparse-0.4.2-py3-none-any.whl (4

In [26]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pprint import pprint
from django.db import models
from data.models import * 
import warnings
import pandas as pd  # for dataframe, suddenly not working
import numpy as np
from tqdm import tqdm
from transformers import *

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

loading configuration file /root/.cache/torch/sentence_transformers/GPL_bioasq-1m-distilbert-tas-b-gpl-self_miner/config.json
Model config DistilBertConfig {
  "_name_or_path": "/root/.cache/torch/sentence_transformers/GPL_bioasq-1m-distilbert-tas-b-gpl-self_miner/",
  "activation": "gelu",
  "architectures": [
    "DistilBertModel"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "vocab_size": 30522
}

loading weights file /root/.cache/torch/sentence_transformers/GPL_bioasq-1m-distilbert-tas-b-gpl-self_miner/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertModel.

All the weights of DistilBertMod

In [19]:
model_STs = {1:"GPL/bioasq-1m-distilbert-tas-b-gpl-self_miner", # used for tasks like clustering or semantic search
                  2:'pritamdeka/S-BioBert-snli-multinli-stsb',
                  3:'paraphrase-MiniLM-L6-v2', #semantic text similarity
                  4:'pritamdeka/S-Biomed-Roberta-snli-multinli-stsb', #biomed ML 
                  5:'GPL/bioasq-1m-distilbert-tas-b-gpl-self_miner',
                  6:'all-mpnet-base-v2'}
model1 = SentenceTransformer(model_STs[1])




loading configuration file /Users/agikajanaku/.cache/torch/sentence_transformers/GPL_bioasq-1m-distilbert-tas-b-gpl-self_miner/config.json
Model config DistilBertConfig {
  "_name_or_path": "/Users/agikajanaku/.cache/torch/sentence_transformers/GPL_bioasq-1m-distilbert-tas-b-gpl-self_miner/",
  "activation": "gelu",
  "architectures": [
    "DistilBertModel"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "vocab_size": 30522
}

loading weights file /Users/agikajanaku/.cache/torch/sentence_transformers/GPL_bioasq-1m-distilbert-tas-b-gpl-self_miner/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertM

# **Drug - sildenafil** (FDA - EMA)

### Section Level similarity - Adverse reaction comparison

In [20]:
# commented sections in the list belongs to the document from query section, hence commented. Not to compare with sections from the same document.

sections = ['']
# Dataframe of Sections
df = pd.DataFrame(({'Sections':sections}))

In [21]:
df

,Sections
0,


In [22]:
def convert_overall_text_to_embedding(df, column_name): # Can be used if required. pass the list instead of dataframe
    '''
    column_name : name of column for which embedding are to be created.
    df : dataframe of sections
    '''
    # The list of all the embeddings
    embeddings = []
    
    # Get overall text data
    overall_text_data = df[column_name].values
    
    # Loop over all the descriptions and get the embeddings
    for sec in tqdm(overall_text_data):
        
        # Get the embedding 
        # embedding = convert_single_abstract_to_embedding(sciBERT_tokenizer, model, abstract)
        embedding = model1.encode(sec)
        #add/append it to the list
        embeddings.append(embedding)
        
    print("\nEmbedding conversion Done!")
    print("\ncreating new column..")
    df["embeddings"] = embeddings
    df["embeddings"] = df["embeddings"].apply(lambda emb: np.array(emb))
    # Reshaping for cosine similarity
    df["embeddings"] = df["embeddings"].apply(lambda emb: emb.reshape(1, -1))
    print("New column 'embedding' created")
    
    return df

df_embeddings = convert_overall_text_to_embedding(df, "Sections") # Uncomment later.

100%|██████████| 1/1 [00:01<00:00,  1.11s/it]


Embedding conversion Done!

creating new column..
New column 'embedding' created


In [54]:
df_embeddings

,Sections,embeddings
0,ADVERSE REACTIONS CLINICAL TRIALS VIAGRA was a...,"[[-0.2575023, 0.122261666, 0.12960626, -0.1444..."
1,In the table below all medically important adv...,"[[-0.23999327, 0.34252346, 0.10279646, -0.1405..."
2,Contraindications \n \nHypersensitivity to the...,"[[-0.060211223, 0.31949767, -0.0067299907, -0...."
3,4.9 Overdose \n In single dose volunteer stud...,"[[0.25022274, 0.24948505, 0.404247, -0.2125837..."
4,\n4.4 Special warnings and precautions for us...,"[[0.07692921, 0.12295768, -0.008425954, -0.329..."


In [55]:
def get_top_N_articles_cosine(query_text, dataFrame, column_name,top_N=5):
    """
    Retrieve top_N (5 is default value) articles similar to the query
    data : dataframe with text and embeddings.
    """
    revevant_cols = [column_name, "cos_sim"]
    query_vect = model1.encode(query_text).reshape(1, -1)
    # Run similarity Search
    dataFrame["cos_sim"] = dataFrame["embeddings"].apply(lambda x: cosine_similarity(query_vect, x))
    dataFrame["cos_sim"] = dataFrame["cos_sim"].apply(lambda x: x[0][0])
    
 
    # Sort Cosine Similarity Column in Descending Order 
    
    most_similar_articles = dataFrame.sort_values(by='cos_sim', ascending=False)[0:top_N+1]
    
    return most_similar_articles[revevant_cols]

In [56]:

get_top_N_articles_cosine(df_embeddings.Sections[0], df_embeddings,"Sections",5)


,Sections,cos_sim
0,ADVERSE REACTIONS CLINICAL TRIALS VIAGRA was a...,1.000000
1,In the table below all medically important adv...,0.881713
3,4.9 Overdose \n In single dose volunteer stud...,0.866781
4,\n4.4 Special warnings and precautions for us...,0.832301
2,Contraindications \n \nHypersensitivity to the...,0.820712


### Sub Section Level Text Similarity - Subsection e.g. Eye

In [57]:
sub_sections = ['']

df = pd.DataFrame(({'Sub_sections':sub_sections}))


In [12]:
df = convert_overall_text_to_embedding(df, "Sub_sections")
get_top_N_articles_cosine(df.Sub_sections[0], df,'Sub_sections',5)

100%|██████████| 18/18 [00:01<00:00, 10.12it/s]



Embedding conversion Done!

creating new column..
New column 'embedding' created


,Sub_sections,cos_sim
0,"Abnormal Vision: Mild and transient, predomina...",1.000000
4,Eye disorders Visual colour Lacrimation N...,0.861098
16,"**Visual colour distortions: Chloropsia, Chrom...",0.835933
6,"Cardiac Tachycardia, Sudden cardiac deat...",0.830118
3,Nervous system disorders Headache Dizziness ...,0.821216
17,"***Lacrimation disorders: Dry eye, Lacrimal di...",0.797844


 Index 4 and 16 both talk about the adverse reactions about the EYE. 

In [77]:
adverse_section = sub_sections

In [78]:
sentence_embeddings = model1.encode(adverse_section)

In [81]:
for sentence, embedding in zip(adverse_section, sentence_embeddings):
    #print("Sentence:", sentence) # can use this is were to sentence by sentence instead of by subsection
    print("Subsection:", sentence)
    print("Embedding:", embedding)
    print("")

Subsection: Abnormal Vision: Mild and transient, predominantly color tinge to vision, but also increased sensitivity to light or blurred vision. In these studies, only one patient discontinued due to abnormal vision. Other adverse reactions occurred at a rate of greater than 2%, but equally 
Embedding: [ 2.02898681e-01  2.04962835e-01 -2.13255644e-01 -1.07203633e-01
  2.93843985e-01  1.17938884e-01  1.04306722e-02  2.38141462e-01
 -4.12481070e-01 -3.57655764e-01 -1.73626602e-01 -4.49683666e-02
 -2.76933998e-01  1.52003393e-01  2.71133264e-03  9.01957452e-01
  5.47254086e-01  5.17998040e-01 -5.28675854e-01  3.33138742e-02
  7.08307922e-02  1.02632418e-01 -2.40241569e-02  6.62938878e-02
 -2.10668921e-01 -5.70562109e-02  2.42672682e-01  4.90509756e-02
 -1.85263390e-03  3.88548360e-03  6.61305711e-02 -1.57455862e-01
  4.16978039e-02  4.20062393e-02 -3.52761522e-02  2.25640923e-01
 -4.64855969e-01 -8.99461061e-02 -1.94338664e-01 -1.71761468e-01
 -1.56278968e-01 -1.05358772e-01 -5.28337108e-

# **Drug - Abiraterone** (FDA vs EMA)

### Comparing Adverse reactions

In [62]:
Section_EMA = ['']

section_FDA = ['']


In [63]:
# Compare adverse sections
import re
section_FDA_new = []
for item in section_FDA:
  item = re.sub(r"\n\s+"," ",item)
  section_FDA_new.append(item)
Section_EMA_new = []
for item in Section_EMA:
  item = re.sub(r"\n\s+|\n+","",item)
  Section_EMA_new.append(item)

In [64]:
Sections = section_FDA_new + Section_EMA_new

In [65]:
Sections

[' 1 INDICATIONS\xa0AND USAGE Abiraterone acetate tablets\xa0are indicated in combination with prednisone  for the treatment of patients with Metastatic castration-resistant prostate cancer (CRPC) Abiraterone  acetate tablets are a CYP17 inhibitor indicated in combination with prednisone for the treatment of  patients with  metastatic castration-resistant prostate cancer (CRPC).',
 " 2 DOSAGE\xa0AND ADMINISTRATION Metastatic castration-resistant prostate cancer:  Abiraterone acetate tablets\xa0 1,000 mg orally once daily with prednisone 5 mg orally  twice  daily. (2.1)  Patients receiving\xa0abiraterone  acetate tablets should also receive a gonadotropin-releasing hormone (GnRH) analog concurrently or should have  had bilateral orchiectomy.\xa0Abiraterone acetate tablets must be taken on an empty stomach with water at least 1  hour before or 2 hours after a meal. Do not crush or chew tablets. (2.3)  Dose Modification:  For patients with  baseline moderate hepatic impairment (Child-Pugh

In [66]:
df = pd.DataFrame(({'Sections':Sections}))

In [67]:
df

,Sections
0,1 INDICATIONS AND USAGE Abiraterone acetate t...
1,2 DOSAGE AND ADMINISTRATION Metastatic castra...
2,4 CONTRAINDICATIONS Pregnancy Abiraterone ac...
3,6 ADVERSE REACTIONS The following are discuss...
4,\n10 OVERDOSAGE Human experience of overdose w...
5,\n8 USE IN SPECIFIC POPULATIONS Females and Ma...
6,4.8 Undesirable effects Summary of the safety...


In [68]:
df_embeddings = convert_overall_text_to_embedding(df, "Sections") # Uncomment later.
df_embeddings

100%|██████████| 7/7 [00:03<00:00,  2.06it/s]


Embedding conversion Done!

creating new column..
New column 'embedding' created


,Sections,embeddings
0,1 INDICATIONS AND USAGE Abiraterone acetate t...,"[[0.06759025, 0.043458976, -0.07615974, -0.098..."
1,2 DOSAGE AND ADMINISTRATION Metastatic castra...,"[[-0.007014582, 0.50157875, 0.102054246, -0.08..."
2,4 CONTRAINDICATIONS Pregnancy Abiraterone ac...,"[[0.028791023, 0.37241715, 0.1416486, -0.11306..."
3,6 ADVERSE REACTIONS The following are discuss...,"[[-0.088344015, 0.395775, -0.050313246, -0.039..."
4,\n10 OVERDOSAGE Human experience of overdose w...,"[[0.20400827, 0.13262163, 0.032169376, -0.1897..."
5,\n8 USE IN SPECIFIC POPULATIONS Females and Ma...,"[[-0.06632709, 0.29709446, -0.003847693, -0.17..."
6,4.8 Undesirable effects Summary of the safety...,"[[-0.1228118, 0.27050048, -0.037973646, 0.0365..."


In [69]:
 # query document which is a section from Epar explaining Undesirable effect or adverse reactions.
df_embeddings.Sections[6]

'4.8  Undesirable effects Summary of the safety profile In an analysis of adverse reactions of composite Phase 3 studies with abiraterone acetate, adverse reactions that were observed in ≥10% of patients were peripheral oedema, hypokalaemia, hypertension, urinary tract infection, and alanine aminotransferase increased and/or aspartate aminotransferase increased. Other important adverse reactions include, cardiac disorders, hepatotoxicity, fractures, and allergic alveolitis. 7 Abiraterone acetate  may cause hypertension, hypokalaemia and fluid retention as a pharmacodynamic consequence of its mechanism of action. In Phase 3 studies, anticipated mineralocorticoid adverse reactions were seen more commonly in patients treated with abiraterone acetate than in patients treated with placebo: hypokalaemia 18 % vs. 8 %, hypertension 22 % vs. 16 % and fluid retention (peripheral oedema) 23 % vs. 17 %, respectively. In patients treated with abiraterone acetate versus patients treated with placebo

In [70]:
get_top_N_articles_cosine(df_embeddings.Sections[6], df_embeddings,"Sections",6)

,Sections,cos_sim
6,4.8 Undesirable effects Summary of the safety...,1.000000
3,6 ADVERSE REACTIONS The following are discuss...,0.919173
5,\n8 USE IN SPECIFIC POPULATIONS Females and Ma...,0.880252
2,4 CONTRAINDICATIONS Pregnancy Abiraterone ac...,0.860819
4,\n10 OVERDOSAGE Human experience of overdose w...,0.858810
1,2 DOSAGE AND ADMINISTRATION Metastatic castra...,0.837857
0,1 INDICATIONS AND USAGE Abiraterone acetate t...,0.825943


Index 6, Undesirable effects from epar are very well matching with Adverse reactions from FDA, index 3.

### Comparing Method of Administration

In [71]:
Section_EMA = ['']

section_FDA = ['']


In [72]:
# Compare adverse sections and fertility and pregnancy.
import re
section_FDA_new = []
for item in section_FDA:
  item = re.sub(r"\n\s+"," ",item)
  section_FDA_new.append(item)
Section_EMA_new = []
for item in Section_EMA:
  item = re.sub(r"\n\s+|\n+","",item)
  Section_EMA_new.append(item)

In [73]:
Sections = section_FDA_new + Section_EMA_new
Sections

[' 1 INDICATIONS\xa0AND USAGE Abiraterone acetate tablets\xa0are indicated in combination with prednisone  for the treatment of patients with Metastatic castration-resistant prostate cancer (CRPC) Abiraterone  acetate tablets are a CYP17 inhibitor indicated in combination with prednisone for the treatment of  patients with  metastatic castration-resistant prostate cancer (CRPC).',
 " 2 DOSAGE\xa0AND ADMINISTRATION Metastatic castration-resistant prostate cancer:  Abiraterone acetate tablets\xa0 1,000 mg orally once daily with prednisone 5 mg orally  twice  daily. (2.1)  Patients receiving\xa0abiraterone  acetate tablets should also receive a gonadotropin-releasing hormone (GnRH) analog concurrently or should have  had bilateral orchiectomy.\xa0Abiraterone acetate tablets must be taken on an empty stomach with water at least 1  hour before or 2 hours after a meal. Do not crush or chew tablets. (2.3)  Dose Modification:  For patients with  baseline moderate hepatic impairment (Child-Pugh

In [74]:
df

,Sections,embeddings,cos_sim
0,1 INDICATIONS AND USAGE Abiraterone acetate t...,"[[0.06759025, 0.043458976, -0.07615974, -0.098...",0.825943
1,2 DOSAGE AND ADMINISTRATION Metastatic castra...,"[[-0.007014582, 0.50157875, 0.102054246, -0.08...",0.837857
2,4 CONTRAINDICATIONS Pregnancy Abiraterone ac...,"[[0.028791023, 0.37241715, 0.1416486, -0.11306...",0.860819
3,6 ADVERSE REACTIONS The following are discuss...,"[[-0.088344015, 0.395775, -0.050313246, -0.039...",0.919173
4,\n10 OVERDOSAGE Human experience of overdose w...,"[[0.20400827, 0.13262163, 0.032169376, -0.1897...",0.858810
5,\n8 USE IN SPECIFIC POPULATIONS Females and Ma...,"[[-0.06632709, 0.29709446, -0.003847693, -0.17...",0.880252
6,4.8 Undesirable effects Summary of the safety...,"[[-0.1228118, 0.27050048, -0.037973646, 0.0365...",1.000000


In [75]:
df = pd.DataFrame(({'Sections':Sections}))
df_embeddings = convert_overall_text_to_embedding(df, "Sections") # Uncomment later.
get_top_N_articles_cosine(df_embeddings.Sections[6], df_embeddings,"Sections",6)

100%|██████████| 7/7 [00:03<00:00,  2.04it/s]



Embedding conversion Done!

creating new column..
New column 'embedding' created


,Sections,cos_sim
6,4.2 Posology and method of administration Thi...,1.000000
1,2 DOSAGE AND ADMINISTRATION Metastatic castra...,0.896064
5,\n8 USE IN SPECIFIC POPULATIONS Females and Ma...,0.860336
0,1 INDICATIONS AND USAGE Abiraterone acetate t...,0.847556
4,\n10 OVERDOSAGE Human experience of overdose w...,0.833933
3,6 ADVERSE REACTIONS The following are discuss...,0.832294
2,4 CONTRAINDICATIONS Pregnancy Abiraterone ac...,0.823573


Index 6, Posology and method of administration from epar are very well matching with DOSAGE AND ADMINISTRATION from FDA, index 1.